In [151]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, time
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import random
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn

In [152]:
df = pd.read_excel('../data/GNP_Aerial_counting_1969_2022.xlsx')

In [153]:
empty_cols = ['MALE', 'CALVES'] #columns that are empty
zero_cols = ['LINE2002', 'LINE2012', 'COLLAR', 'CONSERVANC', 'SANCTUARY'] #columns that are > 80% just 0s
drop_cols = ['NOTES'] # other columns to drop
df.drop(columns=empty_cols, inplace=True)
df.drop(columns=zero_cols, inplace=True)
df.drop(columns=drop_cols, inplace=True)

In [154]:
df['TIME'] = df['TIME'].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second if pd.notna(x) else x)
df['TIME'] = df['TIME'].fillna(0)

In [155]:
df['TYPE'] = df['TYPE'].map({'Fixed-wing': 0, 'Helicopter': 1})

In [135]:
#zero_count = (df['COUNT_DAY'] == 0).sum()
#print(zero_count / df.shape[0])

In [156]:
df['DATE'] = df['DATE'].apply(lambda t: t.day if isinstance(t, datetime) else np.nan)

def process_date(val):
    if pd.isna(val):
        return np.nan
    elif isinstance(val, str):
        return float(val.split('/')[1])
    elif isinstance(val, datetime):
        return val.day
    else:
        return float(val)

df['DATE'] = df['DATE'].apply(process_date)

In [157]:
month_mapping = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4,
    'May': 5, 'June': 6, 'July': 7, 'August': 8,
    'September': 9, 'October': 10, 'November': 11, 'December': 12
}

df['MONTH'] = df['MONTH'].map(month_mapping)
df['MONTH'] = pd.to_numeric(df['MONTH'], errors='coerce')

In [158]:
df['lat_lag1'] = df.groupby('SPECIES')['LATITUDE'].shift(1)
df['lat_lag2'] = df.groupby('SPECIES')['LATITUDE'].shift(2)


df['lon_lag1'] = df.groupby('SPECIES')['LONGITUDE'].shift(1)
df['lon_lag2'] = df.groupby('SPECIES')['LONGITUDE'].shift(2)


df['number_lag1'] = df.groupby('SPECIES')['NUMBER'].shift(1)
df['number_lag2'] = df.groupby('SPECIES')['NUMBER'].shift(2)


In [159]:
df['SPECIES'] = df['SPECIES'].str.lower()
df['STRATUM'] = df['STRATUM'].str.lower()
df = pd.get_dummies(df, columns=['SPECIES', 'STRATUM'])

In [140]:
'''
correlation_matrix = df.corr()
plt.figure(figsize=(25, 25)) 
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm")
plt.title("Correlation Matrix Heatmap")
plt.show()
'''

'\ncorrelation_matrix = df.corr()\nplt.figure(figsize=(25, 25)) \nsns.heatmap(correlation_matrix, annot=True, cmap="coolwarm")\nplt.title("Correlation Matrix Heatmap")\nplt.show()\n'

In [141]:
# df.to_csv('GNB1969-2022.csv', index=False)

### Non-DL Model

#### Run Cross-Val

In [160]:
# Get Train Test Split
train_df = df[df['COUNT'] != 2022]
test_df = df[df['COUNT'] == 2022]

#fillna with mean
train_df = train_df.fillna(train_df.mean())
test_df = test_df.fillna(test_df.mean())

X_train = train_df.drop(columns=['ID', 'LATITUDE', 'LONGITUDE', 'NUMBER'])
y_train = train_df[['NUMBER', 'LATITUDE', 'LONGITUDE']]
X_test = test_df.drop(columns=['ID', 'LATITUDE', 'LONGITUDE', 'NUMBER'])
y_test = test_df[['NUMBER', 'LATITUDE', 'LONGITUDE']]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#### Train on Full Dataset (Excluding 2022 for testing)

In [161]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler



# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train.values)
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.FloatTensor(y_test.values)

# Reshape input to be [samples, time steps, features]
X_train_tensor = X_train_tensor.unsqueeze(1)
X_test_tensor = X_test_tensor.unsqueeze(1)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define the LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout= 0.2)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Instantiate the model
input_size = X_train.shape[1]
hidden_size = 64
num_layers = 2
output_size = 3  # Number of target variables

model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

# Training loop
num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    X_test_tensor = X_test_tensor.to(device)
    predictions = model(X_test_tensor)
    test_loss = criterion(predictions, y_test_tensor.to(device))
    print(f'Test Loss: {test_loss.item():.4f}')

# Convert predictions back to numpy for further analysis if needed
predictions = predictions.cpu().numpy()

Epoch [10/100], Loss: 23.7240
Epoch [20/100], Loss: 9.7731
Epoch [30/100], Loss: 9.6855
Epoch [40/100], Loss: 11.0686
Epoch [50/100], Loss: 4.2876
Epoch [60/100], Loss: 22.6931
Epoch [70/100], Loss: 4.3442
Epoch [80/100], Loss: 27.9471
Epoch [90/100], Loss: 13.8867
Epoch [100/100], Loss: 8.9911
Test Loss: 55.4840


In [167]:
with torch.no_grad():
    actual = y_test_tensor.numpy()
    y_preds = model(X_test_tensor)
    #y_preds[:, 0] = torch.round(y_preds[:, 0])
    predicted = y_preds.detach().numpy()
    
    mse = mean_squared_error(actual, predicted)
    print(f"Mean Squared Error: {mse}")

    mae = mean_absolute_error(actual, predicted)
    print(f"Mean Absolute Error: {mae}")    

    r2 = r2_score(actual, predicted)
    print(f"R² Score: {r2}")

print(predicted)

Mean Squared Error: 55.483638763427734
Mean Absolute Error: 1.541730523109436
R² Score: -0.24870991706848145
[[  1.591094  -18.854141   34.446045 ]
 [  3.5158355 -18.827131   34.45874  ]
 [  1.9997168 -18.85842    34.46708  ]
 ...
 [  6.813117  -18.893501   34.501186 ]
 [  4.32167   -18.86376    34.50691  ]
 [  3.5166614 -18.859398   34.49637  ]]


In [163]:
df_actual = pd.DataFrame(actual)
df_predicted = pd.DataFrame(predicted)

In [147]:
df_actual.head(5)

,0,1,2
0,1.0,-18.860001,34.174301
1,1.0,-18.855499,34.168701
2,2.0,-18.820101,34.150700
3,1.0,-18.819401,34.150902
4,4.0,-18.814800,34.152401


In [ ]:
df_predicted.head(5)

,0,1,2
0,0.535073,-18.878510,34.498390
1,3.314152,-18.827133,34.444683
2,2.388991,-18.843895,34.466145
3,2.146640,-18.852396,34.481922
4,2.553080,-18.831341,34.446682


In [164]:
mse = mean_squared_error(actual[0], predicted[0])
print(f"Mean Squared Error NUMBER: {mse}")

mae = mean_absolute_error(actual[0], predicted[0])
print(f"Mean Absolute Error NUMBER: {mae}")    

r2 = r2_score(actual[0], predicted[0])
print(f"R² Score NUMBER: {r2}")

Mean Squared Error NUMBER: 0.14109037816524506
Mean Absolute Error NUMBER: 0.28956571221351624
R² Score NUMBER: 0.9997051954269409


In [165]:
mse = mean_squared_error(actual[1], predicted[1])
print(f"Mean Squared Error LATITUDE: {mse}")

mae = mean_absolute_error(actual[1], predicted[1])
print(f"Mean Absolute Error LATITUDE: {mae}")    

r2 = r2_score(actual[1], predicted[1])
print(f"R² Score LATITUDE: {r2}")

Mean Squared Error LATITUDE: 2.1381185054779053
Mean Absolute Error LATITUDE: 0.9447475075721741
R² Score LATITUDE: 0.9955310821533203


In [166]:
mse = mean_squared_error(actual[2], predicted[2])
print(f"Mean Squared Error LONGITUDE: {mse}")

mae = mean_absolute_error(actual[2], predicted[2])
print(f"Mean Absolute Error LONGITUDE: {mae}")    

r2 = r2_score(actual[2], predicted[2])
print(f"R² Score LONGITUDE: {r2}")

Mean Squared Error LONGITUDE: 0.033854804933071136
Mean Absolute Error LONGITUDE: 0.11832714080810547
R² Score LONGITUDE: 0.9999287128448486
